In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!yes | unzip /content/gdrive/MyDrive/151B_data/new_train.zip > /dev/null
!yes | unzip /content/gdrive/MyDrive/151B_data/new_val_in.zip > /dev/null

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os, os.path 
import numpy 
import pickle
from glob import glob


"""Change to the data folder"""
new_path = "/content/new_train"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [4]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
train_dataset  = ArgoverseDataset(data_path=new_path)

In [5]:
batch_sz = 100
import numpy as np
import random

def my_collate_train(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """

    batch_inp = []
    batch_out = []

    for scene in batch:
      agent = scene['agent_id']
      target = 0
      for x in range(len(scene['track_id'])):
        if scene['track_id'][x][0] == agent:
          target = x
      inp = [scene['p_in'][target], scene['v_in'][target]]
      out = [scene['p_out'][target], scene['v_out'][target]]
      batch_inp.append(inp)
      batch_out.append(out)

    # scene level #####################
    # batch_inp = []
    # batch_out = []
    # for scene in batch:
    #   mask = scene['car_mask'].flatten()==1
    #   # print(np.count_nonzero(mask))
    #   inp = [scene['p_in'][mask], scene['v_in'][mask]]
    #   out = [scene['p_out'][mask], scene['v_out'][mask]]
    #   batch_inp.append(inp)
    #   batch_out.append(out)
    ####################################


    inp = torch.LongTensor(batch_inp)
    out = torch.LongTensor(batch_out)
    return [inp, out]

def my_collate_train_multiple(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """

    batch_inp = []
    batch_out = []

    for scene in batch:
      agent = scene['agent_id']
      target = 0
      for x in range(len(scene['track_id'])):
        if scene['track_id'][x][0] == agent:
          target = x
      inp = [scene['p_in'][target], scene['v_in'][target]]
      out = [scene['p_out'][target], scene['v_out'][target]]

      other_in = np.zeros((5,2,19,2)) # need to permute to 2,5,19,2 later
      other_out = np.zeros((5,2,30,2))

      other_in[0] = inp
      other_out[0] = out

      mask = scene['car_mask'].flatten()==1
      mask = np.delete(mask,target)
      mask = np.where(mask == True)[0]

      if(len(mask)>=4):
        temp = random.sample(mask.tolist(),4)
        for i in range(len(temp)):
          other_in[i+1] = [scene['p_in'][temp[i]], scene['v_in'][temp[i]]]
          other_out[i+1] = [scene['p_out'][temp[i]], scene['v_out'][temp[i]]]
      else:
        for i in range(len(mask)):
          other_in[i+1] = [scene['p_in'][mask[i]], scene['v_in'][mask[i]]]
          other_out[i+1] = [scene['p_out'][mask[i]], scene['v_out'][mask[i]]]
        for i in range(4-len(mask)):
          other_in[i+1+len(mask)] = inp
          other_out[i+1+len(mask)] = out

      batch_inp.append(other_in.tolist())
      batch_out.append(other_out.tolist())

    # scene level #####################
    # batch_inp = []
    # batch_out = []
    # for scene in batch:
    #   mask = scene['car_mask'].flatten()==1
    #   # print(np.count_nonzero(mask))
    #   inp = [scene['p_in'][mask], scene['v_in'][mask]]
    #   out = [scene['p_out'][mask], scene['v_out'][mask]]
    #   batch_inp.append(inp)
    #   batch_out.append(out)
    ####################################


    inp = torch.LongTensor(batch_inp)
    inp = inp.permute(2,0,1,3,4) # p/v, batch, cars, points, x/y
    out = torch.LongTensor(batch_out)
    out = out.permute(2,0,1,3,4) 
    return [inp, out]

def my_collate_val(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """

    inp = [[scene['p_in'], scene['v_in']] for scene in batch]
    mask = [scene['car_mask'] for scene in batch]

    inp = torch.LongTensor(inp)
    mask = torch.LongTensor(mask)
    return [inp, mask]



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Trajectory(nn.Module):

    def __init__(self):
        super(Trajectory, self).__init__()

        self.p_in = nn.Linear(2,32) # N, 19, 2
        self.v_in = nn.Linear(2,32) # N, 19, 2
        
        self.decoder_p = nn.LSTM(64, 128, 1) # input 19, N, 64 output 1, N, 128
        self.decoder_v = nn.LSTM(64, 128,1) # input 19, N, 64 output 1, N, 128

        self.p_out = nn.Linear(128,2)
        self.v_out = nn.Linear(128,2)

    def forward(self, p, v):
        batch = p.shape[0]
        x_p = self.p_in(p)
        x_v = self.v_in(v)

        x = torch.cat((x_p,x_v),dim=2)
        x = x.permute(1,0,2)

        x_p,_ = self.decoder_p(x)
        x_v,_ = self.decoder_v(x)

        x_p = x_p[-1]
        x_v = x_v[-1]
      
        x_p = self.p_out(x_p)
        x_v = self.v_out(x_v)

        return x_p , x_v

In [6]:
'''
Recursive per point
'''

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Trajectory(nn.Module):

    def __init__(self):
        super(Trajectory, self).__init__()

        self.p_in = nn.Linear(2,32) # N, 19, 2
        self.v_in = nn.Linear(2,32) # N, 19, 2
        
        self.encoder = nn.LSTM(64, 64, 1) # input 19, N, 64 output 1, N, 64 

        self.decoder_p = nn.LSTM(64, 64, 1) # input 30, N, 64 output 30, N, 64
        self.decoder_v = nn.LSTM(64, 64,1) # input 30, N, 64 output 30, N, 64

        self.recurse_p =  nn.Linear(2,32)
        self.recurse_v = nn.Linear(2,32)
 
        self.p_out = nn.Linear(64,2)
        self.v_out = nn.Linear(64,2)

    def forward(self, p, v):
        batch = p.shape[0]
        x_p = self.p_in(p)
        x_v = self.v_in(v)

        x = torch.cat((x_p,x_v),dim=2)
        x = x.permute(1,0,2)

        _,(state_h) = self.encoder(x)

        temp_p = torch.zeros(30,batch,2).to(device) # output of p
        temp_v = torch.zeros(30,batch,2).to(device)
       
        decoder_p_inp = torch.unsqueeze(x[-1],0)
        decoder_v_inp = torch.unsqueeze(x[-1],0)
        state_p = state_h
        state_v = state_h

        for i in range(30):
          x_p,state_p = self.decoder_p(decoder_p_inp, state_p)
          x_v,state_v = self.decoder_v(decoder_v_inp, state_v)

          x_p = self.p_out(x_p)
          x_v = self.v_out(x_v)

          temp_p[i] = x_p
          temp_v[i] = x_v

          x_p = self.recurse_p(x_p)
          x_v = self.recurse_v(x_v)

          x = torch.cat((x_p,x_v),dim=2)

          decoder_p_inp = x
          decoder_v_inp = x

        temp_p = temp_p.permute(1,0,2)
        temp_v = temp_v.permute(1,0,2)
        return temp_p, temp_v

In [ ]:
# don't run

import numpy as np
model = Trajectory().double()
print(model)

p = np.zeros((4,19,2))
v = np.zeros((4,19,2))
p_torch = torch.tensor(p)
v_torch = torch.tensor(v)

output = model(p_torch,v_torch)
print(len(output))
print(output[0].shape)

Trajectory(
  (p_in): Linear(in_features=2, out_features=32, bias=True)
  (v_in): Linear(in_features=2, out_features=32, bias=True)
  (encoder): LSTM(64, 64)
  (decoder_p): LSTM(64, 64)
  (decoder_v): LSTM(64, 64)
  (recurse_p): Linear(in_features=2, out_features=32, bias=True)
  (recurse_v): Linear(in_features=2, out_features=32, bias=True)
  (p_out): Linear(in_features=64, out_features=2, bias=True)
  (v_out): Linear(in_features=64, out_features=2, bias=True)
)
2
torch.Size([4, 30, 2])


In [7]:
def zeroNormNP(array, size=19, pos=np.array([-1,-1])): # array is N,19,2
  temp = []
  if( np.array_equal(pos, np.array([-1,-1]))):
    temp = array[:,0]
  else:
    temp = pos # should be N,2
  temp = np.expand_dims(temp,1)
  temp = np.repeat(temp,size,1)

  return (array-temp)/200., array[:,0]

def zeroNormTor(array, size=19, pos=torch.tensor([-1,-1])): # array is N,19,2
  temp = []
  if( np.array_equal(pos, torch.tensor([-1,-1]))):
    temp = array[:,0]
  else:
    temp = pos # should be N,2
  temp = torch.unsqueeze(temp,1)
  temp = torch.repeat_interleave(temp,size,1)
  return (array-temp)/200., array[:,0]

In [ ]:
import time 

batch_sz = 100
train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train_multiple, num_workers=2)

train_size = int(0.8 * len(train_loader.dataset))
val_size = len(train_loader.dataset) - train_size
train_data, val_data = torch.utils.data.random_split(train_loader.dataset, [train_size, val_size])

print("\nLENGTH OF TRAIN LOADER DATASET:", len(train_loader.dataset))
print("LENGTH OF TRAIN DATA:", len(train_data), "\nLENGTH OF VAL DATA:", len(val_data))

train_data = DataLoader(train_data, batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train_multiple, num_workers=2)
val_data = DataLoader(val_data, batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train_multiple, num_workers=2)
model = Trajectory().to(device)

p_criteria = nn.MSELoss()
v_criteria = nn.MSELoss()

my_optim = torch.optim.Adam(model.parameters())

# decayRate = 0.999
# my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=my_optim, gamma=decayRate)

epoch = 20 # takes around 20 epochs to converge
number = 5 # number of cars in each 

for t in range(epoch):

  model.train()
  epoch_loss = 0
  start = time.time()
  for i_batch, sample_batch in enumerate(train_data):
    inp, out = sample_batch
    batch = inp.shape[1]

    p_in, orig = zeroNormTor(inp[0].reshape(batch*number,19,2).to(device))
    v_in = inp[1].reshape(batch*number,19,2).to(device)/85.
    p_out,_ = zeroNormTor(out[0].reshape(batch*number,30,2).to(device),30, orig)
    v_out = out[1].reshape(batch*number,30,2).to(device)/85.

    sample = p_in.shape[1]
    multiplier = p_out.shape[1]

    p = torch.cat([p_in, p_out], dim=1)
    v = torch.cat([v_in, v_out], dim=1)

    p_list = []
    for i in range(multiplier):
      pred = model(p[:,i:i+sample], v[:,i:i+sample])

      loss = 0
      
      p_loss = torch.sqrt(p_criteria(pred[0], p[:,i+sample]))
      if(len(p_list)==0):
        p_list = torch.unsqueeze(pred[0],dim=1)
      else:
        p_list = torch.cat([p_list, torch.unsqueeze(pred[0],dim=1)],dim=1)

      v_loss = torch.sqrt(v_criteria(pred[1], v[:,i+sample]))

      loss = p_loss + v_loss
      my_optim.zero_grad()
      loss.backward()
      my_optim.step()

    p_loss = torch.sqrt(p_criteria(p_list, p_out))
    epoch_loss += p_loss

  print("Training Loss: ", t, epoch_loss.item(), time.time() - start)
  
  model.eval()
  epoch_loss = 0
  
  for i_batch, sample_batch in enumerate(val_data):
    inp, out = sample_batch
    batch = inp.shape[1]

    p_in, orig = zeroNormTor(inp[0].reshape(batch*number,19,2).to(device))
    v_in = inp[1].reshape(batch*number,19,2).to(device)/85.
    p_out,_ = zeroNormTor(out[0].reshape(batch*number,30,2).to(device),30, orig)
    v_out = out[1].reshape(batch*number,30,2).to(device)/85.

    sample = p_in.shape[1]
    multiplier = 30
    p = p_in
    v = v_in

    for i in range(multiplier):
      pred = model(p[:,i:i+sample], v[:,i:i+sample])

      p = torch.cat([p, torch.unsqueeze(pred[0],dim=1)],dim=1)
      v = torch.cat([v, torch.unsqueeze(pred[1],dim=1)],dim=1)

    p_loss = torch.sqrt(p_criteria(p[:,19:], p_out))
    epoch_loss +=p_loss.item()
  print("Validation Loss:", epoch_loss)
  epoch_loss = 0
  


In [ ]:
# train on multiple cars per
import time

batch_sz = 100
train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train_multiple, num_workers=2)

train_size = int(0.8 * len(train_loader.dataset))
val_size = len(train_loader.dataset) - train_size
train_data, val_data = torch.utils.data.random_split(train_loader.dataset, [train_size, val_size])

print("\nLENGTH OF TRAIN LOADER DATASET:", len(train_loader.dataset))
print("LENGTH OF TRAIN DATA:", len(train_data), "\nLENGTH OF VAL DATA:", len(val_data))

train_data = DataLoader(train_data, batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train_multiple, num_workers=2)
val_data = DataLoader(val_data, batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train_multiple, num_workers=2)
model = Trajectory().to(device)

my_optim = torch.optim.Adam(model.parameters(), lr=0.001)

# decayRate = 0.999    
# my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=my_optim, gamma=decayRate)

epoch = 30 # takes around 20 epochs to converge
number = 5 # number of cars in each 

best_val = 100

for i in range(epoch):

  model.train()
  epoch_loss = 0
  start = time.time()
  
  # train_loop = tqdm(enumerate(train_data), total=len(train_data))

  for i_batch, sample_batch in enumerate(train_data):

    inp, out = sample_batch
    batch = inp.shape[1]

    p_in, orig = zeroNormTor(inp[0].reshape(batch*number,19,2).to(device))
    v_in = inp[1].reshape(batch*number,19,2).to(device)/85.
    p_out,_ = zeroNormTor(out[0].reshape(batch*number,30,2).to(device),30, orig)
    v_out = out[1].reshape(batch*number,30,2).to(device)/85.

    pred = model(p_in, v_in)


    loss = 0
    p_criteria = nn.MSELoss()
    p_loss = torch.sqrt(p_criteria(pred[0], p_out))

    v_criteria = nn.MSELoss()
    v_loss = torch.sqrt(v_criteria(pred[1], v_out))

    loss = p_loss + v_loss
    epoch_loss += p_loss.item()

    my_optim.zero_grad()
    loss.backward()
    my_optim.step()

    # ur tqdm stuff broke
    # train_loop.set_description(f"Train Epoch [{i + 1}/{epoch}]")
    # train_loop.set_postfix(loss = epoch_loss.item()) 
    # my_lr_scheduler.step()
    # if(i>2999):
    #   temp = torch.unsqueeze(orig,1)
      # temp = torch.repeat_interleave(temp,30,1)
      # print((pred[0]*500+temp)[0], (p_out*500+temp)[0])
    # break
  print("Training Loss: ", i, epoch_loss, time.time() - start)
  
  model.eval()
  epoch_loss = 0
  # val_loop = tqdm(enumerate(val_data), total=len(val_data))
  for i_batch, sample_batch in enumerate(val_data):
    inp, out = sample_batch
    batch = inp.shape[1]

    p_in,orig = zeroNormTor(inp[0].reshape(batch*number,19,2).to(device))
    v_in = inp[1].reshape(batch*number,19,2).to(device)/85.
    p_out,_ = zeroNormTor(out[0].reshape(batch*number,30,2).to(device),30, orig)
    v_out = out[1].reshape(batch*number,30,2).to(device)/85.

    pred = model(p_in, v_in)

    loss = 0
    p_criteria = nn.MSELoss()
    p_loss = torch.sqrt(p_criteria(pred[0], p_out))

    epoch_loss += p_loss.item()

    # val_loop.set_description(f"Val.  Epoch [{i + 1}/{epoch}]")
    # val_loop.set_postfix(loss = epoch_loss.item())
  if(epoch_loss< best_val):
    print("saved")
    best_val = epoch_loss

  print("Validation Loss: ", epoch_loss)


LENGTH OF TRAIN LOADER DATASET: 205942
LENGTH OF TRAIN DATA: 164753 
LENGTH OF VAL DATA: 41189
Training Loss:  0 35.65819874498993 472.5862205028534
saved
Validation Loss:  4.988086823374033
Training Loss:  1 21.0319468062371 478.26616406440735
Validation Loss:  5.286611597053707
Training Loss:  2 20.45644376054406 477.5138130187988
saved
Validation Loss:  4.877381956204772
Training Loss:  3 19.9245780473575 478.4371635913849
saved
Validation Loss:  4.77798888925463
Training Loss:  4 19.679950556717813 478.3485782146454
Validation Loss:  4.857072846964002
Training Loss:  5 19.285880726762116 477.92885971069336
saved
Validation Loss:  4.761179870925844
Training Loss:  6 18.9284421633929 478.04212760925293


In [ ]:
import time 

batch_sz = 300
train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate_train, num_workers=2)

model = Trajectory().to(device)

my_optim = torch.optim.Adam(model.parameters())

# decayRate = 0.999
# my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=my_optim, gamma=decayRate)

epoch = 20 # takes around 20 epochs to converge
number = 1 # number of cars in each 
print(len(train_loader.dataset))

p_criteria = nn.MSELoss()
v_criteria = nn.MSELoss()

for i in range(epoch):

  model.train()
  epoch_loss = 0
  start = time.time()
  for i_batch, sample_batch in enumerate(train_loader):
    inp, out = sample_batch
    batch = inp.shape[0]

    p_in = (inp[:,0].reshape(batch*number,19,2).to(device))/4800. # -2400.)/2400.
    v_in = inp[:,1].reshape(batch*number,19,2).to(device)/100.
    p_out = (out[:,0].reshape(batch*number,30,2).to(device))/4800. # -2400.)/2400.
    v_out = out[:,1].reshape(batch*number,30,2).to(device)/100.

    sample = p_in.shape[1]
    multiplier = p_out.shape[1]

    p = torch.cat([p_in, p_out], dim=1)
    v = torch.cat([v_in, v_out], dim=1)

    p_list = []
    for i in range(multiplier):
      pred = model(p[:,i:i+sample], v[:,i:i+sample])

      loss = 0
      
      p_loss = torch.sqrt(p_criteria(pred[0], p[:,i+sample]))
      if(len(p_list)==0):
        p_list = torch.unsqueeze(pred[0],dim=1)
      else:
        p_list = torch.cat([p_list, torch.unsqueeze(pred[0],dim=1)],dim=1)

      v_loss = torch.sqrt(v_criteria(pred[1], v[:,i+sample]))

      loss = p_loss + v_loss
      my_optim.zero_grad()
      loss.backward()
      my_optim.step()


    p_loss = torch.sqrt(p_criteria(p_list, p_out))
    epoch_loss += p_loss

  print("Training Loss: ", i, epoch_loss.item(), time.time() - start)
  
  model.eval()
  epoch_loss = 0
  


205942
Training Loss:  29 1.9498718976974487 343.7735676765442
Training Loss:  29 0.7784473299980164 347.1312220096588
Training Loss:  29 0.5861884951591492 351.60469460487366
Training Loss:  29 0.49096304178237915 348.19951248168945
Training Loss:  29 0.4370681047439575 346.88306307792664
Training Loss:  29 0.39462095499038696 348.0212461948395
Training Loss:  29 0.3679283559322357 346.5364649295807
Training Loss:  29 0.3424796164035797 346.18383526802063
Training Loss:  29 0.3258410096168518 347.2324411869049
Training Loss:  29 0.31043338775634766 346.28952169418335
Training Loss:  29 0.29714950919151306 346.1159317493439
Training Loss:  29 0.285498708486557 346.20251846313477
Training Loss:  29 0.27799952030181885 347.88939690589905
Training Loss:  29 0.2672721743583679 346.81219387054443
Training Loss:  29 0.2621701955795288 347.6236653327942
Training Loss:  29 0.25984466075897217 347.6486961841583
Training Loss:  29 0.2521623969078064 346.34811091423035
Training Loss:  29 0.244770

In [ ]:
model.eval()
p_criteria = nn.MSELoss()

for i_batch, sample_batch in enumerate(train_loader):
  inp, out = sample_batch
  batch = inp.shape[0]

  p_in = (inp[:,0].reshape(batch*number,19,2).to(device))/4800. # -2400.)/2400.
  v_in = inp[:,1].reshape(batch*number,19,2).to(device)/100.
  p_out = (out[:,0].reshape(batch*number,30,2).to(device))/4800. # -2400.)/2400.
  v_out = out[:,1].reshape(batch*number,30,2).to(device)/100.

  sample = p_in.shape[1]
  multiplier = 30
  p = p_in
  v = v_in

  for i in range(multiplier):
    pred = model(p[:,i:i+sample], v[:,i:i+sample])

    p = torch.cat([p, torch.unsqueeze(pred[0],dim=1)],dim=1)
    v = torch.cat([v, torch.unsqueeze(pred[1],dim=1)],dim=1)

  p_loss = torch.sqrt(p_criteria(p[:,19:], p_out))
  print(p_loss)
  break

tensor(0.0054, device='cuda:0', grad_fn=<SqrtBackward>)


In [ ]:
import csv 
model.eval()
temp = []

new_path = "/content/new_val_in"
val_dataset  = ArgoverseDataset(data_path=new_path)

top = []
top.append("ID")
for i in range(60):
  top.append("v"+str(i+1))
temp.append(top)

with torch.no_grad():
    for i in val_dataset:
      row = []
      scene = i['scene_idx']
      agent = i['agent_id']
      target =0
      for x in range(len(i['track_id'])):
        if i['track_id'][x][0] == agent:
          target = x

      p_in = torch.LongTensor(i['p_in'])
      v_in = torch.LongTensor(i['v_in'])

      p_in = (p_in.reshape(60,19,2).to(device))/4800.
      v_in = v_in.reshape(60,19,2).to(device)/100.

      sample = p_in.shape[1]
      multiplier = 30
      p = p_in
      v = v_in

      for i in range(multiplier):
        pred = model(p[:,i:i+sample], v[:,i:i+sample])

        p = torch.cat([p, torch.unsqueeze(pred[0],dim=1)],dim=1)
        v = torch.cat([v, torch.unsqueeze(pred[1],dim=1)],dim=1)

      pred_out = p[:,19:]*4800.
      
      output = pred_out[target]

      row.append(scene)
      row = row + torch.flatten(output).cpu().detach().numpy().tolist()
      temp.append(row)

with open('submission3.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(temp)